Data Obtained from 
https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

Objective is to run a classification Machine learning algorithms

In [43]:
# Import modules for data handling
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Import classification ML algorithms
# https://towardsdatascience.com/top-machine-learning-algorithms-for-classification-2197870ff501
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Import data prep modules
from sklearn.model_selection import train_test_split

Load dataset to view data

In [44]:
bank_data = pd.read_csv('bank.csv', sep=';')
bank_full_data = pd.read_csv('bank-full.csv', sep=';')
columns_one = bank_data.columns
columns_two = bank_full_data.columns
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [45]:
bank_full_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [77]:
# Check to ensure columns are the same
# Using python to check this eliminates all errors on the human side
if (columns_one.all() == columns_two.all()):
    print("true")
# print(bank_data.info())
print(bank_full_data.info())

true
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB
None


In [47]:
# Instead of merging the dataframes we will concat them because there is no one columns that is unique to both dataframes for them to merge on
bank_df = pd.concat([bank_data, bank_full_data], ignore_index=True)
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49732 entries, 0 to 49731
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        49732 non-null  int64 
 1   job        49732 non-null  object
 2   marital    49732 non-null  object
 3   education  49732 non-null  object
 4   default    49732 non-null  object
 5   balance    49732 non-null  int64 
 6   housing    49732 non-null  object
 7   loan       49732 non-null  object
 8   contact    49732 non-null  object
 9   day        49732 non-null  int64 
 10  month      49732 non-null  object
 11  duration   49732 non-null  int64 
 12  campaign   49732 non-null  int64 
 13  pdays      49732 non-null  int64 
 14  previous   49732 non-null  int64 
 15  poutcome   49732 non-null  object
 16  y          49732 non-null  object
dtypes: int64(7), object(10)
memory usage: 6.5+ MB


In [48]:
bank_df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

1 - age (numeric)

2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')

3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)

4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

5 - default: has credit in default? (categorical: 'no','yes','unknown')

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:

8 - contact: contact communication type (categorical: 'cellular','telephone')

9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')

11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:

12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not 
previously contacted)

14 - previous: number of contacts performed before this campaign and for this client (numeric)

15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes

16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)

17 - cons.price.idx: consumer price index - monthly indicator (numeric)

18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)

19 - euribor3m: euribor 3 month rate - daily indicator (numeric)

20 - nr.employed: number of employees - quarterly indicator (numeric)


Output variable (desired target):

21 - y - has the client subscribed a term deposit? (binary: 'yes','no')

In [57]:
# transform the data to be used by the machine learning classification algorithms
bank_df['y_binary'] = bank_df['y'].map({"no": 0, "yes": 1}) 
bank_df['poutcome_num'] = bank_df['poutcome'].map({"failure": 0, "success": 1, "unknown": 2, "other": 3}) 
bank_df['month_num'] = bank_df['month'].map({"jan": 0, "feb": 1, "mar": 2, "apr": 3, "may": 4, "jun": 5, \
     "jul": 6, "aug": 7, "sep": 8, "oct": 9, "nov": 10, "dec": 11}) 
bank_df['contact_num'] = bank_df['contact'].map({"cellular": 0, "telephone": 1, "unknown": 2}) 
bank_df['marital_num'] = bank_df['marital'].map({"single": 0, "divorced": 1, "married": 2}) 
bank_df['education_num'] = bank_df['education'].map({"primary": 0, "secondary": 1, "unknown": 2, "tertiary": 3}) 
bank_df['loan_default_binary'] = bank_df['default'].map({"no": 0, "yes": 1}) 
bank_df['has_housing_num'] = bank_df['housing'].map({"no": 0, "yes": 1, "unknown": 2}) 
bank_df['has_loan_num'] = bank_df['loan'].map({"no": 0, "yes": 1, "unknown": 2}) 

# Create the variables that need to be split for machine learning
features = bank_df[['age', 'campaign', 'pdays', 'poutcome_num', 'month_num', 'contact_num', 'marital_num', 'education_num', 'loan_default_binary', 'has_housing_num', 'has_loan_num' ]]
labels = bank_df['y_binary']


In [50]:
# bank_df['y'] = bank_df['y'].astype('string')
bank_df['day'].value_counts()

20    3009
18    2534
21    2224
17    2130
6     2119
5     2091
14    2043
8     2022
28    2011
7     2007
19    1958
29    1920
15    1877
12    1754
13    1751
30    1734
9     1724
11    1631
4     1584
16    1579
2     1407
27    1234
3     1184
26    1145
23    1041
22     991
25     920
31     702
10     574
24     483
1      349
Name: day, dtype: int64

In [51]:
bank_df['has_loan_num'].value_counts()

0    41797
1     7935
Name: has_loan_num, dtype: int64

In [52]:
bank_df.corr()

,age,balance,day,duration,campaign,pdays,previous,y_binary,poutcome_num,month_num,contact_num,marital_num,education_num,loan_default_binary,has_housing_num,has_loan_num
age,1.000000,0.096564,-0.009880,-0.004399,0.003877,-0.022419,0.000928,0.026939,-0.013309,0.091204,0.025240,0.374853,-0.124146,-0.017895,-0.186225,-0.015299
balance,0.096564,1.000000,0.003348,0.018195,-0.014147,0.003968,0.017243,0.049705,-0.018826,0.095087,-0.025661,0.019351,0.078094,-0.067118,-0.067068,-0.083231
day,-0.009880,0.003348,1.000000,-0.029674,0.162336,-0.093166,-0.052083,-0.026821,0.060982,0.100077,-0.028452,0.006684,0.024082,0.007432,-0.028256,0.009908
duration,-0.004399,0.018195,-0.029674,1.000000,-0.083061,-0.000478,0.002330,0.395099,0.005232,-0.010799,-0.019944,-0.023710,0.000241,-0.010179,0.006088,-0.011781
campaign,0.003877,-0.014147,0.162336,-0.083061,1.000000,-0.089042,-0.035162,-0.072085,0.089902,0.055266,0.018953,0.028005,0.009134,0.014234,-0.021760,0.010603
pdays,-0.022419,0.003968,-0.093166,-0.000478,-0.089042,1.000000,0.461822,0.103665,-0.586881,-0.109217,-0.244674,-0.028627,-0.001032,-0.029652,0.123508,-0.023490
previous,0.000928,0.017243,-0.052083,0.002330,-0.035162,0.461822,1.000000,0.094567,-0.290484,-0.035636,-0.150067,-0.016507,0.023889,-0.018825,0.037042,-0.011729
y_binary,0.026939,0.049705,-0.026821,0.395099,-0.072085,0.103665,0.094567,1.000000,-0.079476,0.019125,-0.147057,-0.065113,0.070985,-0.020336,-0.136070,-0.068381
poutcome_num,-0.013309,-0.018826,0.060982,0.005232,0.089902,-0.586881,-0.290484,-0.079476,1.000000,0.026242,0.192795,0.003118,-0.023667,0.027610,-0.060794,0.013865
month_num,0.091204,0.095087,0.100077,-0.010799,0.055266,-0.109217,-0.035636,0.019125,0.026242,1.000000,-0.175065,0.068755,0.077092,0.014452,-0.173601,0.023176


In [53]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [54]:
# create a list of models to loop through since we will be using more than one
models = [
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=20, random_state=1),
    GaussianNB(),
    SVC(),
    KNeighborsClassifier(n_neighbors=10),
]
for model in models:
    model.fit(X_train, y_train)
    print(f'{model.__class__.__name__}')
    print(f'Train score: {round((model.score(X_train, y_train)*100), 2)}%')
    print(f'Test score: {round((model.score(X_test, y_test)*100), 2)}%')
    print('\n')

DecisionTreeClassifier
Train score: 96.78%
Test score: 86.82%


RandomForestClassifier
Train score: 96.54%
Test score: 88.67%


GaussianNB
Train score: 79.15%
Test score: 79.08%


SVC
Train score: 88.34%
Test score: 88.21%


KNeighborsClassifier
Train score: 89.58%
Test score: 88.56%




So out of the models I just ran, I am inclined to go with the RandomForestClassifier. WIth a train score being second best at 96.54% we have the highest test score with 88.67%.
The data we put in is as follows:


bank_df['age'] = {"numeric"}

bank_df['campaign'] = {"numeric"}

bank_df['pdays'] = {"numeric"}

bank_df['poutcome_num'] = {"failure": 0, "success": 1, "unknown": 2, "other": 3}

bank_df['month_num'] = {"jan": 0, "feb": 1, "mar": 2, "apr": 3, "may": 4, "jun": 5, \
     "jul": 6, "aug": 7, "sep": 8, "oct": 9, "nov": 10, "dec": 11}

bank_df['contact_num'] = {"cellular": 0, "telephone": 1, "unknown": 2} 

bank_df['marital_num'] = {"single": 0, "divorced": 1, "married": 2}

bank_df['education_num'] = {"primary": 0, "secondary": 1, "unknown": 2, "tertiary": 3}

bank_df['loan_default_binary'] = {"no": 0, "yes": 1}

bank_df['has_housing_num'] = {"no": 0, "yes": 1, "unknown": 2}

bank_df['has_loan_num'] = {"no": 0, "yes": 1, "unknown": 2}

In [73]:
# Now to test this out of myself
RFC = RandomForestClassifier(n_estimators=20, random_state=1)
mock_info = np.array([[27., 10., 999., 3., 9., 0., 2., 1., 0., 1., 1.]])



In [76]:


RFC.fit(X_train, y_train)
print(f'{RFC.__class__.__name__}')
print(f'Train score: {round((RFC.score(X_train, y_train)*100), 2)}%')
print(f'Test score: {round((RFC.score(X_test, y_test)*100), 2)}%')
print('\n')
print(RFC.predict(mock_info))
print(RFC.predict_proba(mock_info))


RandomForestClassifier
Train score: 96.54%
Test score: 88.67%


[0]
[[0.8 0.2]]


C:\Users\pc\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pc\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Based on the information I put in about myself, I have an 80% chance of not subscribing and a 20% of subscribing to this marketing campaign. 